In [78]:
OBSIDIAN_VAULT_URL = "./link_demo"

In [79]:
# recursively find all markdown file names in the vault directory

import os
import re
import glob

def find_markdown_files(directory: str) -> list[str]:
    """
    Find all markdown files in the directory and its subdirectories.

    Args:
        directory (str): The directory to search for markdown files.

    Returns:
        list[str]: A list of markdown files found in the directory and its subdirectories.
    """
    markdown_files = glob.glob(directory + '/**/*.md', recursive=True)
    return markdown_files

In [93]:
from pprint import pprint

# test find_markdown_files

files = find_markdown_files(OBSIDIAN_VAULT_URL)
pprint(files[:10])

['./link_demo/Obsidian vs. Roam Research.md',
 './link_demo/Appending Drafts to Dropbox Daily Notes in Obsidian.md',
 './link_demo/The Power of the Local Graph.md',
 './link_demo/Splitting Notes in Obsidian.md',
 './link_demo/Timeblocking in Obsidian.md',
 './link_demo/Connecting Notes & Bidirectional Linking.md',
 './link_demo/Keyboard Hotkeys and The Command Pallette.md',
 './link_demo/Using Templates in Obsidian.md',
 './link_demo/Changing Your Obsidian Theme.md',
 "./link_demo/Nick Milo's Mindfulness Monday Interview.md"]


In [81]:
# extract just the filename, without extension, from a filepath 

def extract_filename(filepath: str) -> str:
    """
    Extract the filename from the filepath without the extension.

    Args:
        filepath (str): The filepath to extract the filename from.

    Returns:
        str: The filename without the extension.
    """
    filename = os.path.basename(filepath) # get the filename from the filepath
    filename = os.path.splitext(filename)[0] # remove the extension from the filename
    return filename

In [87]:
# test extract_filename on 10 files 

for file in files[:10]:
    filename = extract_filename(file)
    filename = filename.replace(' ', '_')
    print(filename)


Obsidian_vs._Roam_Research
Appending_Drafts_to_Dropbox_Daily_Notes_in_Obsidian
The_Power_of_the_Local_Graph
Splitting_Notes_in_Obsidian
Timeblocking_in_Obsidian
Connecting_Notes_&_Bidirectional_Linking
Keyboard_Hotkeys_and_The_Command_Pallette
Using_Templates_in_Obsidian
Changing_Your_Obsidian_Theme
Nick_Milo's_Mindfulness_Monday_Interview


In [83]:
from collections import defaultdict
import re

class Reference:
    def __init__(self, filename, line_number, line_text):
        self.filename = filename
        self.line_number = line_number
        self.line_text = line_text

def find_references(files: list[str]) -> dict[str, list[Reference]]:
    """
    Find all references to each filename in the list of files.

    Args:
        files (list[str]): A list of files to search for references.

    Returns:
        dict[str, list[Reference]]: A dictionary of filenames and a list of references to the filename.
    """
    references = defaultdict(list)  # Use defaultdict with list as the default factory
    for file in files:
        filename = extract_filename(file)
        for search_file in files:
            if file == search_file:
                continue  # skip the file itself
            with open(search_file, 'r') as f:
                for line_number, line in enumerate(f, start=1):
                    # skip blank lines
                    if len(line.strip()) == 0:
                        continue
                    if re.search(rf'(?:\s|^|\*){filename}(?:\s|$|\*)', line):
                        reference = Reference(search_file, line_number, line)
                        references[filename].append(reference)
    return references

In [84]:
# test find_references 

references = find_references(files)
print(references)

defaultdict(<class 'list'>, {'Obsidian vs. Roam Research': [<__main__.Reference object at 0x106a3b7f0>], 'Welcome': [<__main__.Reference object at 0x106aa2ee0>], 'Writing': [<__main__.Reference object at 0x106aa27c0>, <__main__.Reference object at 0x106963df0>, <__main__.Reference object at 0x1069632b0>, <__main__.Reference object at 0x106aa7910>], 'Productivity': [<__main__.Reference object at 0x106a7f280>], 'create a link': [<__main__.Reference object at 0x106978160>, <__main__.Reference object at 0x106a7f3d0>], 'Journaling': [<__main__.Reference object at 0x1068b1790>]})


In [85]:
# given a dictionary of references, print each reference, with the line and the filename highlighted in green 

def print_references(references: dict[str, list[Reference]]):
    """
    Print each reference in the references dictionary, with the filename and line highlighted in green.

    Args:
        references (dict[str, list[Reference]]): A dictionary of filenames and a list of references to the filename.
    """
    for filename, refs in references.items():
        print(f"\n\033[92m{filename}\033[0m")
        for ref in refs:
            ref_text = ref.line_text.strip()
            # prepend the line number to the line text in light blue
            ref_text = f"\033[94m{ref.line_number}\033[0m: {ref_text}"
            ref_text = ref_text.replace(filename, f"\033[92m{filename}\033[0m")
            # append the filename in parentheses in light gray
            ref_text += f" (\033[90m{ref.filename}\033[0m)"
            print(ref_text)


In [86]:
# test print_references

print_references(references)


Obsidian vs. Roam Research
24: - `&file=Articles%2FObsidian%20vs.%20Roam%20Research` specifies the file path in the vault. *Articles* is the folder, and *Obsidian vs. Roam Research* is the file name. *%2F* represents the slash in the directory path, and *%20* represents the spaces in the file name. (./link_demo/Callback URLs in Obsidian.md)

Welcome
3: And an unlinked mention: Welcome (./link_demo/create a link.md)

Writing
70: - [ ] 15:00 Writing (./link_demo/Timeblocking in Obsidian.md)
54: ## Obsidian Community Plugins for an Upgraded Writing Experience (./link_demo/Turning Obsidian into the Perfect Writing App.md)
48: This is my favorite plugin, hands down. I can't recommend it enough. And if you want to read more about how I use it, check out [[Mike's Obsidian-Based Writing Workflow]]. (./link_demo/A Few of Our Favorite Obsidian Plugins.md)
104: It's pretty simple as far as plugins go, but it makes Obsidian an even better writing tool. If you want to know more, check out [[Turnin

## TODO 

- Truncate context line output to a couple words on either side of the reference 
- Add interface for creating the explicit links 